# Eve hands-on

Day 1:
- Write a low-level IR (LIR) (close to the generated code)
- Write a code generator for C++ from the IR

Day 2:
- Start from a high-level IR (HIR) representing a DSL
- Write a pass: extent analysis
- Lower from HIR to LIR

# Day 1

## Problem

```cpp
void fun(std::array<int,2> domain, field& out, field const& in) {
    assert();
    assert();
    for(std::size_t i = 1; i < domain[0] - 1; ++i) {
        for(std::size_t j = 1; j < domain[1] - 1; ++j) {
            out[i][j] = -4. * in[i][j]
                + in[i-1][j] + in[i+1][j]
                + in[i][j-1] + in[i][j-1];
        }
    }
}
```

## Task

1. Write the low-level IR in Eve to represent stencil codes like the one in the example above. Don't try to implement the general case, but just nodes required to solve the concrete problem.
2. Write a code generator which generates simple C++ code as in the example above.

In [ ]:
# Define the IR here

from eve import Node, Str
from typing import List

class Expr(Node):
    pass

class Literal(Expr):
    value: Str

class BinaryOp(Expr):
    left: Expr
    right: Expr
    op: Str

In [ ]:
# Programmatically construct a concrete IR for the Laplacian example

from devtools import debug

# always use keyword arguments for fields
binop = BinaryOp(left=Literal(value="1"), right=Literal(value="1"), op="+")
debug(binop)


In [ ]:
# Code generator

from eve.codegen import FormatTemplate as as_fmt, TemplatedGenerator

class LIR_to_cpp(TemplatedGenerator):
    Literal = as_fmt("{value}")
    BinaryOp = as_fmt("({left}{op}{right})")

In [ ]:
# Call the generator

import os
from eve.codegen import format_source

cppcode = LIR_to_cpp.apply(binop)
formatted_code = format_source("cpp", cppcode, style="LLVM")
print(formatted_code)

output_file = "generated.hpp"
with open(output_file, "w+") as output:
    output.write(formatted_code)

In [ ]:
!g++ driver.cc -o lap
!./lap